### Next steps:

This Catalyst library is a bit of a letdown.  Switch to Pytorch Lightning instead https://github.com/PyTorchLightning/pytorch-lightning

Set up running of sequences of values using Optuna:
https://github.com/optuna/optuna

Here's an example script with an integration of the two: https://github.com/optuna/optuna/blob/master/examples/pytorch_lightning_simple.py

TODO:
1. Set up the batch dimension to be one chunk per batch.  Batchsize = n_chunks
1. Get a per-chunk mask (by spatial batch), and use to focus training loss only on pixels with signal (reduces false "negative" signal outside surfzone)
1. Develop method for measuring wave propogation speed in image coordinates,  then find the flattened homography that makes this as constant as possible.
1. Train again with wave equation now that wavespeed is more uniform
1. Infer an accumulated wave speed field (squared slowness field) in the flattened coordinates for wavefunction normalization with non-uniform propogation speed. 

#### First get the basic Tensorflow Lightning training loop working

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet

tb_logger = pl.loggers.TensorBoardLogger('logs/', name="basic_tests")

trainer = pl.Trainer(
    logger=tb_logger,
    limit_val_batches=1,
    max_epochs=10,
    gpus=1 if torch.cuda.is_available() else None,
    )
    
model = LitSirenNet(hidden_features=128, hidden_layers=3, first_omega_0=3, hidden_omega_0=0.3, squared_slowness=3.0,
                    steps_per_vid_chunk=20, learning_rate=1e-4)
trainer.fit(model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 50 K   | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


1

### Now get optuna optimization trials working

In [ ]:
from surfbreak.studies import run_waveform_hyperparam_search

study = run_waveform_hyperparam_search(n_trials=100, timeout=4*60*60, max_epochs=10, logdir='logs', model_folder='results')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
/home/erik/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



/home/erik/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



### Osurfbreak values from this run:
Setting both omegas to ~0.3, with 128 hidden features gives the best results. 